In [1]:
import pickle
import numpy as np
import numpy.ma as ma

# ДЗ 10 от 19.10.20

## Задание

Реализовать языковую модель информационного поиска для поиска предложений в Википедии для $\lambda_1 = 0.5$  и $\lambda_2 = 0.9$



## Загружаем лемматизированные тексты из предыдущего задания

In [2]:
queries = [
    'Пейзаж с выжженной равниной увековечил образ вечного города.',
    'Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.',
    'Трезубец на жёлто-синем фоне — это флаг Барбадоса.'
]

with open('../Task_4/lemmatized.pkl', 'rb') as output_file:
    docs, original_sentences, queries_processed = pickle.load(output_file)

Убираем односимвольные токены и документы без токенов

In [3]:

docs = [[t for t in tokens if len(t)>1] for tokens in docs]

original_sentences = [sent for tokens, sent in zip(docs, original_sentences) if len(tokens)>0]
docs = [tokens for tokens in docs if len(tokens)>0]

Векторизуем документы и запросы

In [18]:
def vectorize(docs, vocab=None):
    '''
    docs - list of lists of tokens
    vocab - set of tokens
    Returns count array with shape (N x V), where N - number of documents and V - size of vocab
    '''
    if vocab is None:
        vocab = set([token for sentence in docs for token in sentence if len(token)>0])
    vectorized = np.zeros((len(docs),len(vocab)))

    token_ind_dict = {tok: ind for ind, tok in dict(enumerate(vocab)).items()}

    for doc_ind, doc in enumerate(docs):
        for token in doc:
            try:
                token_id = token_ind_dict[token]
                vectorized[doc_ind][token_id] += 1
            except KeyError:
                pass
    return vectorized, token_ind_dict

In [44]:
vocab = set([token for sentence in docs for token in sentence if len(token)>1])
doc_vec, token_ind_dict = vectorize(docs, vocab)
queries_vec, _ = vectorize(queries_processed, vocab)

Функция вычисления вероятностей для одного запроса:

In [124]:
def get_lm_probs(query_vec, doc_vec, lmbd):

    query_terms = np.where(query_vec)[0]
    corpus_freqs = doc_vec.sum(axis=0)
    corpus_size = corpus_freqs.sum()
    doc_sizes = doc_vec.sum(axis=1)

    global_probs = corpus_freqs[query_terms] / corpus_size
    local_probs = (doc_vec[:,query_terms].T / doc_sizes).T

    doc_probs = np.prod(global_probs * lmbd + local_probs * (1-lmbd), axis=1)

    return doc_probs

### Результаты для случая $\lambda_1 = 0.5$

In [125]:
for i, query in enumerate(queries):

    lm_probs = get_lm_probs(queries_vec[i], doc_vec, lmbd=0.5)

    print(f"Query:\n{query}\n")
    print("Results:\n  Weight     | Sentence")

    for ind in lm_probs.argsort()[:-4:-1]:
        print(f"({lm_probs[ind]:.4e})  {original_sentences[ind]}")
        # print(f"With weight {query_results[ind,i]:.6f}")
    print("------\n")

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight     | Sentence
(2.0586e-04)  рим стали часто называть вечным лат
(3.6289e-05)  также рим называют городом на семи холмах
(3.6289e-05)  roma ˈroːma  столица и крупнейший город италии
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight     | Sentence
(1.1438e-10)  биологическое оружие
(1.7276e-11)  в 1879 году он был избран губернатором кентукки от демократической партии
(6.2974e-12)  luke pryor blackburn 16 июня 1816  14 сентября 1887  американский врач и политик губернатор штата кентукки в 18791883 годах
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight     | Sentence
(9.3245e-11)  флаг барбадоса  официальный символ государства барбадос
(3.3265e-13)  посейдон со своей женой богиней амфитритой и сыном тритоном обитают в роскошном дворце на дне моря в окружении нереид гиппокампов и других обитателей м

### Результаты для случая $\lambda_2 = 0.9$

In [126]:
for i, query in enumerate(queries):

    lm_probs = get_lm_probs(queries_vec[i], doc_vec, lmbd=0.9)

    print(f"Query:\n{query}\n")
    print("Results:\n  Weight    | Sentence")

    for ind in lm_probs.argsort()[:-4:-1]:
        print(f"({lm_probs[ind]:.4e})  {original_sentences[ind]}")
        # print(f"With weight {query_results[ind,i]:.6f}")
    print("------\n")

Query:
Пейзаж с выжженной равниной увековечил образ вечного города.

Results:
  Weight    | Sentence
(7.7582e-05)  рим стали часто называть вечным лат
(1.6538e-05)  также рим называют городом на семи холмах
(1.6538e-05)  roma ˈroːma  столица и крупнейший город италии
------

Query:
Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.

Results:
  Weight    | Sentence
(3.3223e-11)  биологическое оружие
(6.8081e-12)  в 1879 году он был избран губернатором кентукки от демократической партии
(3.9720e-12)  его обвинили в заговоре с целью убийства и в нарушении нейтралитета канады но освободили под залог в 8000 долларов
------

Query:
Трезубец на жёлто-синем фоне — это флаг Барбадоса.

Results:
  Weight    | Sentence
(2.3839e-11)  флаг барбадоса  официальный символ государства барбадос
(2.9532e-13)  посейдон со своей женой богиней амфитритой и сыном тритоном обитают в роскошном дворце на дне моря в окружении нереид гиппокампов и других обитателей моря мчится п

Для двух случаев абсолютные величины вероятностей меняются значительно, однако порядок для топ-3 документов сохраняется.

Для данного случая, результаты по запросам выглядят заметно более осмысленными, если сравнивать с результатами TF-iDF модели